In [ ]:
from pyrosm import OSM
import osmnx as ox
import network_bands

import geopandas as gpd
import pandas as pd
import uuid
import os
import matplotlib.pyplot as plt


In [ ]:
#https://build.nisra.gov.uk/en/custom/data?d=HOUSEHOLD&v=DZ21&v=HH_LIFESTAGE_AGG15



In [ ]:
#set base directory for data file paths.
base_dir = os.getcwd()

In [ ]:
# create network graph and edges.
base_road_path = f'{base_dir}\\testEnvironment\\Data\\belfast_super_trimmed.osm.pbf'
G, nodes, edges = network_bands.load_osm_network(file_path=base_road_path, network_type='driving', graph_type='networkx')

In [ ]:
#Start locations
start_locations = pd.read_csv(f'{base_dir}\\testEnvironment\\Data\\libraries_belfast_2024.csv')

#Ensure data is converted to a dataframe
start_locations_gdf = network_bands.csv_to_gdf(start_locations, 'X COORDINATE', 'Y COORDINATE', 29902, 4326)

In [ ]:
#Create the network

start_locations_nearest_node = network_bands.nearest_node_and_name(G, start_locations=start_locations_gdf,  location_name = 'Static Library Name')
#Create service areas for each distance. Remember, these overlap.
#input custom distances as a list.
search_distances = [1000,2000,3000]

alpha_areas = network_bands.single_source_polygon(nearest_node_dict=start_locations_nearest_node, graph=G, search_distances=search_distances,
                                                  alpha_value=500, weight = 'distance', progress=True)

In [ ]:
#import pointer data
pointer = gpd.read_file(f'{base_dir}\\testEnvironment\\Data\\pointer_trimmed_for_trimmed_library.shp')


# assign each house a uuid.
pointer['uuid'] = pointer.apply(lambda index: uuid.uuid4(), axis =1)


In [ ]:
#Ensures that the pointer and start location CRS are the same (should be 4326 if using osm data)
if pointer.crs != start_locations_gdf.crs:
    pointer = pointer.to_crs(start_locations_gdf.crs)



In [ ]:
data_zones = gpd.read_file(f'{base_dir}\\testEnvironment/Data/DZ2021.shp')
data_zones['LGD2014_nm'].unique()
belfast_zones = data_zones[data_zones['LGD2014_nm'] == 'Belfast']

In [ ]:
belfast_zones

In [ ]:
fig, ax = plt.subplots(figsize=(12,12))

edges.plot(ax=ax, zorder=2)
pointer.plot(ax=ax, color='black', zorder = 12, markersize=2)
start_locations_gdf.plot(ax=ax, color='red', zorder=13, markersize=150)
